<a href="https://colab.research.google.com/github/handsome1201/Failure/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bar graph & Circle graph

In [28]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install pyyaml==5.1
# !pip install torch== 1.11.0 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install -q pytesseract
!pip install transformers
!pip install datasets
!sudo apt-get install tesseract-ocr
import cv2
import numpy as np
from datasets import load_dataset
import pytesseract
from PIL import Image
!pip install easyocr
import easyocr

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-nmvq1xtp
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-nmvq1xtp
  Resolved https://github.com/facebookresearch/detectron2.git to commit ff53992b1985b63bd3262b5a36167098e3dada02
  Preparing metadata (setup.py) ... done
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


### 그래프 종류 판별

In [29]:
def detect_graph_orientation(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=250, maxLineGap=15)

    if lines is not None:
        vertical_lines = []
        horizontal_lines = []

        for line in lines:
            x1, y1, x2, y2 = line[0]
            if abs(x1 - x2) < 5:  # Check if the line is vertical
                vertical_lines.append(line)
            elif abs(y1 - y2) < 5:  # Check if the line is horizontal
                horizontal_lines.append(line)

        if len(vertical_lines) > len(horizontal_lines):
            return 1  # "Vertical"
        elif len(horizontal_lines) > len(vertical_lines):
            return 2  # "Horizontal"
        else:
            return 0

### input image

In [41]:
image_path = '/content/drive/MyDrive/자료/한글 수직2.png'

In [42]:
img = cv2.imread(image_path)

orientation = detect_graph_orientation(img)

reader = easyocr.Reader(['en', 'ko'], gpu=False)

result = reader.readtext(image_path)
img = Image.open(image_path)

if orientation == 1: #수직 그래프
    img_array = np.array(img)
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    result = reader.readtext(gray)
    for r in result:
        x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
        text_data = r[1]
    x_values = []
    y_values = []
    grouped_values = {}
    grouped_values_list=[]

    for r in result:
        x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
        x_values.append(x)
        y_values.append(y)

    grouped_indices = []  # 묶인 값들의 인덱스를 저장할 리스트

    for i, x in enumerate(x_values):
        if i in grouped_indices:  # 이미 묶인 값인 경우 건너뜁니다
            continue

        group = [result[i][1]]  # 현재 값과 묶일 값들을 저장할 리스트
        group_indices = [i]  # 현재 값과 묶일 값들의 인덱스를 저장할 리스트

        # 값의 차이가 25 이하인 값을 찾아서 묶어줍니다
        for j in range(i+1, len(x_values)):
            if abs(x_values[j] - x) <= 25:
                group.append(result[j][1])
                group_indices.append(j)


        if len(group) > 1:  # 묶인 값들이 2개 이상인 경우에만 처리합니다
            for value in group[1:]:
                grouped_values[value] = group[0]
                grouped_indices.extend(group_indices[1:])  # 묶인 값들의 인덱스를 추가합니다
            grouped_values_list.append(group)  # group 값을 grouped_values_list에 추가합니다


    for key, value in grouped_values.items():
        print(f"key: {key}, value: {value}")

    print("수직")

elif orientation == 2: #수평 그래프
    img_array = np.array(img)
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    result = reader.readtext(gray)
    for r in result:
        x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
        text_data = r[1]
    x_values = []
    y_values = []
    grouped_values = {}
    grouped_values_list = []

    for r in result:
        x, y = min(list(_[0] for _ in r[0])), min(list(_[1] for _ in r[0]))
        x_values.append(x)
        y_values.append(y)

    grouped_indices = []  # 묶인 값들의 인덱스를 저장할 리스트

    for i, y in enumerate(y_values):
        if i in grouped_indices:  # 이미 묶인 값인 경우 건너뜁니다
            continue

        group = [result[i][1]]  # 현재 값과 묶일 값들을 저장할 리스트
        group_indices = [i]  # 현재 값과 묶일 값들의 인덱스를 저장할 리스트

        # 값의 차이가 25 이하인 값을 찾아서 묶어줍니다
        for j in range(i+1, len(y_values)):
            if abs(y_values[j] - y) <= 25:
                group.append(result[j][1])
                group_indices.append(j)

        if len(group) > 1:  # 묶인 값들이 2개 이상인 경우에만 처리합니다
            for value in group[1:]:
                grouped_values[value] = group[0]
                grouped_indices.extend(group_indices[1:])  # 묶인 값들의 인덱스를 추가합니다
            grouped_values_list.append(group)  # group 값을 grouped_values_list에 추가합니다


    for key, value in grouped_values.items():
        print(f"value: {value}, key: {key}")

    print("수직")


# else:

#   재민님께서 넣으실 코드


key: 4월, value: 50
key: 스마트 스토어, value: 상반기
key: 상품주문 수량, value: 상반기
key: X 출처논 이렇계 하단에 작게 넣어주세요, value: 상반기
key: 그래프 디자인올 유지하면서 수정하려면, value: 상반기
key: [우클렉]-[데이터편집올 이용하세요, value: 상반기
key: 2월, value: 40
key: 5월, value: 40
key: 3월, value: 35
key: 6월, value: 32
key: 1월, value: 20
수직
